In [1]:
import numpy as np
import pandas as pd
import os
from utils.read_data import read_data
num_cols = ['Gestation', 'Rectime', 'Age', 'Parity', 'Abortions', 'Weight',
            'Hypertension', 'Diabetes', 'Placental_position',
            'Bleeding_first_trimester', 'Bleeding_second_trimester', 'Funneling', 'Smoker']


In [2]:
sig_file=[]
data_file=[]

In [3]:
for file in os.listdir("dataset"):
    file_path=os.path.join("dataset",file)
    if file.endswith(".hea"):
        data_file.append(file_path)
    else:
        sig_file.append(file_path)


In [4]:
all_metadata=[]
all_label=[]
for file in data_file:
    metadata,label=read_data(file)
    if metadata != None and label!=None:
        all_metadata.append(metadata)
        all_label.append(label[0])

df=pd.DataFrame(all_metadata)


Cột	Ý nghĩa
Gestation	Tuổi thai tại lúc sinh, tính bằng tuần. Đây là một biến liên tục.
Rectime	Tuổi thai tại thời điểm ghi dữ liệu (recording time), tính bằng tuần.
Age	Tuổi của mẹ tại thời điểm mang thai, tính theo năm.
Parity	Số lần mẹ đã sinh con sống trước đó. Ví dụ, 0 nếu đây là lần sinh đầu tiên.
Abortions	Số lần mẹ bị sảy thai hoặc phá thai trước đó.
Weight	Cân nặng của mẹ, thường tính bằng kg.
Hypertension	Mẹ có bị tăng huyết áp hay không. 0 = không, 1 = có.
Diabetes	Mẹ có bị tiểu đường hay không. 0 = không, 1 = có.
Placental_position	Vị trí nhau thai: ví dụ 1 = phía trước (anterior), 0 = phía sau (posterior).
Bleeding_first_trimester	Chảy máu trong 3 tháng đầu thai kỳ. 0 = không, 1 = có.
Bleeding_second_trimester	Chảy máu trong 3 tháng giữa thai kỳ. 0 = không, 1 = có.
Funneling	Funneling cổ tử cung (cervical funneling) – dấu hiệu nguy cơ sinh non. 0 = không, 1 = có.
Smoker	Mẹ có hút thuốc hay không. 0 = không, 1 = có.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Gestation                  159 non-null    object
 1   Rectime                    159 non-null    object
 2   Age                        159 non-null    object
 3   Parity                     159 non-null    object
 4   Abortions                  159 non-null    object
 5   Weight                     159 non-null    object
 6   Hypertension               138 non-null    object
 7   Diabetes                   138 non-null    object
 8   Placental_position         159 non-null    object
 9   Bleeding_first_trimester   138 non-null    object
 10  Bleeding_second_trimester  138 non-null    object
 11  Funneling                  138 non-null    object
 12  Smoker                     159 non-null    object
dtypes: object(13)
memory usage: 16.3+ KB


In [6]:
df=df[['Rectime','Age','Parity','Abortions','Weight','Bleeding_first_trimester','Bleeding_second_trimester','Smoker']]

In [12]:
df = df.replace('None', np.nan)

In [13]:
df.tail()

,Rectime,Age,Parity,Abortions,Weight,Bleeding_first_trimester,Bleeding_second_trimester,Smoker
154,32.6,25,NaN,NaN,76,no,no,NaN
155,31,37,2,1,65,no,no,no
156,30.6,NaN,NaN,NaN,76,no,no,NaN
157,31.3,20,0,0,80,no,no,no
158,31.1,35,0,NaN,NaN,NaN,NaN,NaN


In [14]:
df.to_csv('du_lieu.csv', index=False, encoding='utf-8')


In [12]:
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in df.columns:
    mode_val = df[col].mode()
    if len(mode_val) > 0:
        df[col] = df[col].fillna(mode_val[0])
    else:
        df[col] = df[col].fillna(0)



In [94]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [106]:
X_values=df.values
y_values=np.array(all_label)

In [107]:
y_values.shape

(159,)

In [128]:
y_test_all = []
y_hat_all = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [129]:
for repeat in range(1):
    for train_index, test_index in kf.split(X_values):
        X_train, X_test = X_values[train_index], X_values[test_index]
        y_train, y_test = y_values[train_index], y_values[test_index]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = LogisticRegression(penalty='l1', solver='saga', max_iter=1000)
        model.fit(X_train_scaled, y_train)

        y_hat = model.predict_proba(X_test_scaled)[:, 1]
        

        y_test_all.extend(y_test)
        y_hat_all.extend(y_hat)

auc = roc_auc_score(y_test_all, y_hat_all)
print(f'Mean AUC over 20x5 folds: {auc:.4f}')

Mean AUC over 20x5 folds: 0.9990
